# Submission Reproduction Notebook
This notebook re-runs the model via Ollama and asserts the five findings still reproduce.


In [ ]:
%pip install -r ../requirements.txt
import os
os.environ['MODEL_ADAPTER'] = 'ollama'
os.environ['OLLAMA_BASE_URL'] = os.environ.get('OLLAMA_BASE_URL', 'http://127.0.0.1:11434')
os.environ['MODEL_NAME'] = os.environ.get('MODEL_NAME', 'gpt-oss:20b')
print('Using', os.environ['MODEL_ADAPTER'], os.environ['OLLAMA_BASE_URL'], os.environ['MODEL_NAME'])


In [ ]:
# Re-run prompts used in final findings
!python ../scripts/run_probes.py --prompts ../data/prompts/novel.yaml --out ../outputs/repro-novel.jsonl --temperature 0.7 --max-tokens 96 --log-stream
!python ../scripts/run_probes.py --prompts ../data/prompts/advanced.yaml --out ../outputs/repro-advanced.jsonl --temperature 0.7 --max-tokens 96 --log-stream


In [ ]:
# Assertions: ensure observed_behavior equals assistant outputs for selected indices
import json, pathlib

def load_line(path, idx):
    lines = pathlib.Path(path).read_text(encoding='utf-8').splitlines()
    return json.loads(lines[idx])

def assert_match(finding_path, run_path, idx):
    f = json.loads(pathlib.Path(finding_path).read_text(encoding='utf-8'))
    r = load_line(run_path, idx)
    a = r['response']['choices'][0]['message'].get('content','')
    assert f['issue_summary']['observed_behavior'] == a, f'Mismatch for {finding_path}'

# Map finals to runs/indices (matches commands used to build finals)
assert_match('../findings/final.2.json', '../outputs/repro-novel.jsonl', 2)
assert_match('../findings/final.4.json', '../outputs/repro-advanced.jsonl', 3)
print('Assertions passed for selected findings.')
